# Getting Started in Jupyter Notebook

## Import the necessary modules

In [ ]:
import logging
import pandas as pd
import numpy as np
from typing import Dict, Any
from pyOFW.ofwConfig import AppType, OFWConfig

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # for offline support

# to display multiple output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Initiate the logging [Optional but recommended]

In [ ]:
logging.basicConfig(
    level=logging.DEBUG,
    # level=logging.INFO,
    format="%(asctime)s.%(msecs)03d %(levelname)s: %(message)s",
    datefmt="%d %H:%M:%S",
)
log = logging.getLogger(__name__)
log.info("Ready to log!")

## Make things ready for Open Flows API

The approach below is using `pyOFW` module which comes with simplified version `OFWConfig` class.

In [ ]:
ofw_config = OFWConfig(AppType.WaterOPS, OFWConfig.wops_install_dir)

## Import OpenFlows related modules

It is important to import these modules after loading the `OpenFlows.dll` which happens in `OFWConfig` file.

In [ ]:
from OpenFlows.Water.Domain import IWaterModel
from OpenFlows.Water import OpenFlowsWater

## Open up the model file

In [ ]:
# Path of the model file to be opened
model_filepath = r"C:\Program Files (x86)\Bentley\WaterGEMS\Samples\Example5.wtg"

logging.info("Opening model...")
model: IWaterModel = OpenFlowsWater.Open(model_filepath)

if model != None:
  logging.info(f"Model opened. Path: {model_filepath}")

## Run a Scenario

In [ ]:
# display the active scenario
scenario = model.ActiveScenario
logging.info(f"Active scenario = ID: {scenario.Id}, Label: {scenario.Label}")

# display all the scenarios in the model
scenarios_text = [f"ID: {scenario.Id}, Label: {scenario.Label}" for scenario in model.Scenarios.Elements()]
logging.info("List of scenarios in the model:")
logging.info("\n".join(scenarios_text))


# check if we have results
has_results = model.ActiveScenario.HasResults
logging.info(f"Active scenario '{model.ActiveScenario.Label}' has results?  {has_results}")

# run the active scenario
logging.info("About to run the simulation")
model.RunActiveScenario()
logging.info("Ran the simulation. ")


# now we should have results
has_results = model.ActiveScenario.HasResults
logging.info(f"Active scenario '{model.ActiveScenario.Label}' has results?  {has_results}")

## Prepare data for plotting/charting/graphing

In [ ]:
# Prepare the datetime (x-axis)
times_in_sec = model.ActiveScenario.TimeStepsInSeconds
df = pd.DataFrame()
df["DateTime"] = pd.to_datetime([f"{model.ActiveScenario.TimeStepToDateTime(t)}" for t in times_in_sec])


# prepare the y-axis
# tank levels
tanks = model.Network.Tanks.Elements()
tank_results: Dict[str, Any] = dict()

for tank in tanks:
    tank_results[tank.Label] =tank.Results.Levels()

# pump flows
pumps = model.Network.Pumps.Elements()
pump_results: Dict[str, Any] = dict()

for pump in pumps:
    pump_results[pump.Label] = pump.Results.Flows()

# combine all data into one
df = df.join(pd.DataFrame(tank_results))
df = df.join(pd.DataFrame(pump_results))

df.head()

## Chart/graph/plot the data in one panel (using secondary axis)

In [ ]:
fig = go.Figure()
chart_name = f"Tank Level & Pump Flow - [Scenario = {model.ActiveScenario.Label}]"

def add_line_chart_h(fig, x_values: pd.Series, y_values: pd.Series, name: str, right_y:bool=False):
    y_axis_name = "y2" if right_y else None
    return fig.add_trace(go.Scatter(x=x_values, y=y_values, name=name,yaxis=y_axis_name))

fig = add_line_chart_h(fig, df[df.columns[0]], df[df.columns[1]], df.columns[1])
fig = add_line_chart_h(fig, df[df.columns[0]], df[df.columns[2]], df.columns[2], True)
fig = add_line_chart_h(fig, df[df.columns[0]], df[df.columns[3]], df.columns[3],True)

fig = fig.update_layout(
        title_text=chart_name,
        yaxis=dict(title="Level"),
        yaxis2=dict(overlaying="y", side="right", position=1.0, title="Flows"),
        margin=dict(l=10, r=10, t=50, b=20),
        hovermode='x', # Enable compare data on hover by default
        showlegend=True,
    )

fig.show()

## Chart/graph/plot the data in two panels (sub-plots, just like in WaterGEMS)

In [ ]:
fig = go.Figure()
chart_name = f"Tank Level & Pump Flow (WaterGEMS Style) - [Scenario = {model.ActiveScenario.Label}]"

fig = make_subplots(
    rows=2,
    cols=1,
    shared_yaxes=False,
    shared_xaxes=True,
    vertical_spacing=0.0,
)

def add_line_chart_v(fig, x_values: pd.Series, y_values: pd.Series, name: str, bottom_panel:bool=False):
    row = 2 if bottom_panel else 1
    return fig.add_trace(go.Scatter(x=x_values, y=y_values, name=name,),row=row, col=1)

fig = add_line_chart_v(fig, df[df.columns[0]], df[df.columns[1]], df.columns[1])
fig = add_line_chart_v(fig, df[df.columns[0]], df[df.columns[2]], df.columns[2], True)
fig = add_line_chart_v(fig, df[df.columns[0]], df[df.columns[3]], df.columns[3], True)

fig = fig.update_layout(
        title_text=chart_name,
        yaxis=dict(title= f"Level ({model.Units.NetworkUnits.Tank.LevelUnit.ShortLabel})"),
        yaxis2=dict(title=f"Flow  ({model.Units.NetworkUnits.Pump.FlowUnit.ShortLabel})"),
        margin=dict(l=30, r=30, t=50, b=20),
        hovermode='x', # Enable compare data on hover by default
        showlegend=True,
        legend=dict(orientation="h",xanchor="center", x=0.5),
    )

fig.show()

## Close the model

In [ ]:
# Disable the comments below to close the model
# if model:
#     model.Close()

## Close the session

If you want to open up the model again, make sure to call the `OFWConfig` as [done above](#Import-OpenFlows-related-modules)

In [ ]:
# Disable the comment below to close the session
# OpenFlowsWater.EndSession()